In [26]:
import pandas as pd
import re

## 生成药物相互关系数据

In [99]:
df_drug_interaction = pd.read_csv("processed/chemical_component_translation.csv").fillna("")
df_drug_interaction = df_drug_interaction[(df_drug_interaction["drug_interaction"] != "") & (df_drug_interaction["cn_chemical_name"] != "")]
df_drug_interaction = df_drug_interaction[["drug_name", "chemical_name", "cn_chemical_name", "drug_interaction"]].drop_duplicates() # .to_csv("processed/label_dataset.txt", sep="\t", index=False, header=False)

In [72]:
df_di = df_drug_interaction.groupby(["cn_chemical_name", "chemical_name"])["drug_interaction"].apply(lambda x: "\r\n".join(x)).reset_index()

In [73]:
chemical_list = []
cn_chemical_list = []
interaction_list = []
for index, row in df_di.iterrows():
    interact_list = list(set(filter(lambda x: x.strip() != "", 
               map(lambda x: re.sub("([\d]+.|\(|\)|（|）|·|\.|【|】|●|[\u2460-\u2499]+)", "", 
                                    re.sub("[，|、|：|﹑]", ",", re.sub("[”|“]", "\"", x))).strip(),
                   re.split(r"[\n|;|。|；]", 
                            row["drug_interaction"])))))
    interact_list = ["{}. {}。".format(i, interact_list[i - 1]) for i in range(1, len(interact_list) + 1)]
    
    chemical_list.append(row["chemical_name"].strip().lower())
    cn_chemical_list.append(row["cn_chemical_name"].strip())
    interaction_list.append("\n".join(interact_list))

df_di = pd.DataFrame({"chemical": chemical_list, "cn_chemical": cn_chemical_list, "interaction": interaction_list})

In [74]:
df_dd = df_drug_interaction.groupby(["cn_chemical_name", "chemical_name"])["drug_name"].apply(lambda x: "\r\n".join(x)).reset_index()
df_dd["chemical_name"] = df_dd["chemical_name"].str.lower()

In [75]:
df_join = pd.merge(df_dd, df_di, how="left", left_on=["chemical_name"], right_on=["chemical"])
df_join = df_join[["chemical", "cn_chemical", "drug_name", "interaction"]]
df_join.to_csv("processed/interaction_data.csv", index=False)

## 通过化合物中文翻译匹配新的药物化合物关系

In [126]:
df_translation = pd.read_csv("processed/chemical_translation.csv", dtype=str).fillna("")
df_all_drug = pd.read_csv("processed/all_drug.csv", dtype=str).fillna("")

In [127]:
translation_dict = dict(zip(list(df_translation["cn_chemical_name"].str.strip().str.lower().values), 
                            list(df_translation["chemical_name"].str.strip().str.lower().values)))

translation_reverse_dict = dict(zip(list(df_translation["chemical_name"].str.strip().str.lower().values), 
                            list(df_translation["cn_chemical_name"].str.strip().str.lower().values)))

In [128]:
# filter no need chemical
no_need_che_list = ["calcium", "carbasalate calcium", "enzymes", "glucose", "inhalants",
                    "platinum", "recombinant", "vitamin c", "vitamin e", "vitamin k", "vitamin k1"]

In [129]:
match_list = []
for index, row in df_all_drug.iterrows():
    m_list = list(filter(lambda x: x[0], [[True, x] if x in row["drug_name"] else [False, x] for x in translation_dict.keys()]))
    if len(m_list) > 0:
        for m in m_list:
            match_list.append([m[1], translation_dict[m[1]], row["drug_name"]])

In [130]:
B_set = set(["{}:{}".format(x[1].strip().lower(), x[2].strip().lower()) for x in match_list])

In [131]:
df_drug_interaction = pd.read_csv("processed/chemical_component_translation.csv").fillna("")
A_set = set(map(lambda x: "{}:{}".format(x[0].strip().lower(), x[1].strip().lower()),
    list(zip(df_drug_interaction["chemical_name"].values, df_drug_interaction["drug_name"].values))))

In [132]:
pd.DataFrame(list(filter(lambda x: x[0] not in no_need_che_list, [[x.split(":")[0], 
 translation_reverse_dict[x.split(":")[0]], x.split(":")[1]]
 for x in B_set - A_set.intersection(B_set)])), columns=["chemical", 
 "cn_chemical", "drug"]).to_csv("processed/new_match_drug_chemical.csv", index=False)

In [133]:
len(A_set)

2856